In [10]:
# Formulas
import numpy as np
import pandas as pd

def as_colvec(x):
    if (x.ndim == 2):
        return x        
    else:
        return np.expand_dims(x, axis=1)
def implied_returns(delta, sigma, w):
    """
Obtain the implied expected returns by reverse engineering the weights
Inputs:
delta: Risk Aversion Coefficient (scalar)
sigma: Variance-Covariance Matrix (N x N) as DataFrame
    w: Portfolio weights (N x 1) as Series
Returns an N x 1 vector of Returns as Series
    """
    ir = delta * sigma.dot(w).squeeze() # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

# Assumes that Omega is proportional to the variance of the prior
def proportional_prior(sigma, tau, p):
    """
    Returns the He-Litterman simplified Omega
    Inputs:
    sigma: N x N Covariance Matrix as DataFrame
    tau: a scalar
    p: a K x N DataFrame linking Q and Assets
    returns a P x P DataFrame, a Matrix representing Prior Uncertainties
    """
    helit_omega = p.dot(tau * sigma).dot(p.T)
    # Make a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)
from numpy.linalg import inv

def bl(w_prior, sigma_prior, p, q,
                omega=None,
                delta=2.5, tau=.02):
    """
# Computes the posterior expected returns based on 
# the original black litterman reference model
#
# W.prior must be an N x 1 vector of weights, a Series
# Sigma.prior is an N x N covariance matrix, a DataFrame
# P must be a K x N matrix linking Q and the Assets, a DataFrame
# Q must be an K x 1 vector of views, a Series
# Omega must be a K x K matrix a DataFrame, or None
# if Omega is None, we assume it is
#    proportional to variance of the prior
# delta and tau are scalars
    """
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vectors
    # How many assets do we have?
    N = w_prior.shape[0]
    # And how many views?
    K = q.shape[0]
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(delta, sigma_prior,  w_prior)
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior  
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require
    # Omega to be inverted (see previous section)
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #     mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
#     sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)
# for convenience and readability, define the inverse of a dataframe
def inverse(d):
    """
    Invert the dataframe by inverting the underlying matrix
    """
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)

def w_msr(sigma, mu, scale=True):
    """
    Optimal (Tangent/Max Sharpe Ratio) Portfolio weights
    by using the Markowitz Optimization Procedure
    Mu is the vector of Excess expected Returns
    Sigma must be an N x N matrix as a DataFrame and Mu a column vector as a Series
    This implements page 188 Equation 5.2.28 of
    "The econometrics of financial markets" Campbell, Lo and Mackinlay.
    """
    w = inverse(sigma).dot(mu)
    if scale:
        w = w/sum(w) # fix: this assumes all w is +ve
    return w


In [5]:
import edhec_risk_kit_206 as erk
import numpy as np
import pandas as pd
ind49_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["2013":]
ind49_mcap = erk.get_ind_market_caps(49, weights=True)["2013":]
inds = ['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']
rho_ = ind49_rets[inds].corr()
vols_ = (ind49_rets[inds].std()*np.sqrt(12))    
sigma_prior_ =  (vols_.T).dot(vols_) * rho_

In [17]:
market_weights = ind49_mcap[inds].iloc[0]/ind49_mcap[inds].iloc[0].sum()
type(market_weights)

pandas.core.series.Series

In [13]:
# Implied Returns
delta = 2.5
implied_R = implied_returns(delta=delta,sigma=sigma_prior_,w=market_weights)
implied_R

Hlth     0.152910
Fin      0.175580
Whlsl    0.201836
Rtail    0.224827
Food     0.158115
Name: Implied Returns, dtype: float64

In [51]:
# cORRECT Impose the subjective relative view that Hlth will outperform Rtail and Whlsl by 3%

q = pd.Series([.03])

p = pd.Series([.0]*len(inds),index=inds)
wealth_p = 1
retail_p = market_weights['Rtail']/(market_weights['Rtail']+market_weights['Whlsl'])
health_p = market_weights['Hlth']/(market_weights['Hlth']+market_weights['Rtail']+market_weights['Whlsl'])

p['Hlth'] = wealth_p
p['Rtail'] = -retail_p
p['Whlsl'] = - (1 - retail_p)
p.round(3
        ).sum

<bound method Series.sum of Hlth     1.000
Fin      0.000
Whlsl   -0.151
Rtail   -0.849
Food     0.000
dtype: float64>

In [34]:
bl_mu

Hlth     0.179994
Fin      0.164283
Whlsl    0.186389
Rtail    0.204020
Food     0.145081
dtype: float64

In [35]:
w_msr(bl_sigma, bl_mu)

Hlth     0.296749
Fin      0.175362
Whlsl   -0.030132
Rtail    0.418845
Food     0.139176
dtype: float64

In [36]:
# 9 . hlth outperform Retail and whlsl by 3%

q = pd.Series([.05])

p = pd.DataFrame([
    {'Hlth': 1, 'Whlsl': -0.5, 'Rtail': -0.5, 'Fin':0,'Food':0}
])

bl_mu, bl_sigma = bl(market_weights,sigma_prior=sigma_prior_,p=p,q=q,tau=.05,delta=2.5)

In [40]:
(bl_mu*100).round(2).sort_values()

Food     14.22
Fin      16.18
Whlsl    18.30
Hlth     18.60
Rtail    19.94
dtype: float64

In [38]:
w_msr(bl_sigma, bl_mu)

Hlth     0.355025
Fin      0.175362
Whlsl   -0.059270
Rtail    0.389707
Food     0.139176
dtype: float64